In [3]:
import pandas as pd
energy = pd.read_excel("E:\Anaconda3\Energy Indicators.xls", header = 1, skiprows = 15, parse_cols= "C:G",
                   na_values = "...", skip_footer=38, thousands=',')
energy = energy[1:]
energy.columns.values[0] = "Country"
energy = energy.rename(columns={'Renewable Electricity Production': '% Renewable'})
energy = energy.rename(columns={'Energy Supply per capita': 'Energy Supply per Capita'})
energy['Country'].replace({"\(.*\)":""}, regex=True, inplace=True)
energy['Country'] = energy['Country'].map(lambda x: x.strip(''))
energy['Country'].replace({"[0-9]+":""}, regex=True, inplace=True)
energy['Country'] = energy['Country'].map(lambda x: x.strip())
energy.replace({"Republic of Korea": "South Korea"}, inplace=True)
energy.replace({"United States of America": "United States"}, inplace=True)
energy.replace({"United Kingdom of Great Britain and Northern Ireland": "United Kingdom"}, inplace=True)
energy.replace({"China, Hong Kong Special Administrative Region": "Hong Kong"}, inplace=True)
energy.set_index('Country', drop=False, inplace=True, verify_integrity=True)

GDP = pd.read_csv('E:\Anaconda3\world_bank.csv', skiprows=4, thousands=',')
GDP['Country Name'].replace({"[0-9]+":""}, regex=True, inplace=True)

GDP.replace({"Korea, Rep.": "South Korea"}, inplace=True)
GDP.replace({"Iran, Islamic Rep.": "Iran"}, inplace=True)
GDP.replace({"Hong Kong SAR, China": "Hong Kong"}, inplace=True)
GDP.set_index('Country Name', drop=True, inplace=True, verify_integrity=True)

ScimEn = pd.read_excel("E:\Anaconda3\scimagojr.xlsx", header = 0,
                   na_values = "...", thousands=',')
df = pd.merge(energy, GDP, how='inner', left_index=True, right_index=True)
df = pd.merge(ScimEn, df, how='inner', left_on='Country', right_index=True)
df = df[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
df.head()
print(len(df) - len(df[df['Rank'] < 16]))

df = df[df['Rank'] < 16]
avgGDP = df[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]\
    .mean(skipna=True, axis=1).sort_values(ascending=False)
df['avg GDP'] = df[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]\
    .mean(skipna=True, axis=1)
df.sort_values('avg GDP',ascending=False, inplace=True)
print(df.loc[5]['2015'] - df.loc[5]['2006'])
print(energy['Energy Supply per Capita'].mean())
energy.sort_values('% Renewable',ascending=False, inplace=True)
print(((energy.iloc[0]).name, energy.iloc[0].get_value('% Renewable')))

ScimEn['Ratio'] = ScimEn['Self-citations']/ScimEn['Citations']
print((ScimEn.iloc[ScimEn['Ratio'].idxmax()]['Country'], ScimEn['Ratio'].max()))

energy['Population'] = energy['Energy Supply']/energy['Energy Supply per Capita']
energy.sort_values('Population',ascending=False, inplace=True)
print(energy.iloc[2].get_value('Country'))

df['Population'] = df['Energy Supply']/df['Energy Supply per Capita']
df['Citable documents per person'] = df['Citable documents']/df['Population']
print(df[['Citable documents per person', 'Energy Supply per Capita']]
      .corr(method='pearson', min_periods=0)['Citable documents per person'][1])
def is_high(x):
    return 1 if (x >= df['% Renewable'].median()) else 0

df['HighRenew'] = df['% Renewable'].apply(is_high)
HighRenew = df[df['HighRenew'] > 0].sort_values('Rank',ascending=False)

ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
df['Continent'] = df.index.map(ContinentDict)
df2 = df.groupby('Continent')['Population'].agg(['size', 'sum','mean','std'])
print(df2)

147
235121322681.0
90.66666666666667
('Bhutan', 100.0)
('French Guiana', 1.0)
United States
0.794001043544


TypeError: 'dict' object is not callable